In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import yaml
import glob
import rawpy
import zivid
import os
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
path = "data"
zdf1 = "20221117_1013_calib_test.zdf"
cam2 = "DSCF9210.RAF" 

with open(os.path.join(path, "calib_result.yaml")) as f:
    calib = yaml.safe_load(f)

k1 = np.array(calib["cameraMatrix1"])
d1 = np.array(calib["distCoeffs1"])
k2 = np.array(calib["cameraMatrix2"])
d2 = np.array(calib["distCoeffs2"])
R = np.array(calib["R"])
T = np.array(calib["T"])
E = np.array(calib["E"])
F = np.array(calib["F"])
RT = np.hstack((R, T))

In [3]:
# load zivid point cloud
with zivid.Application() as app:
    frame = zivid.Frame(os.path.join(path, zdf1))
    point_cloud = frame.point_cloud()
    xyz = point_cloud.copy_data("xyz")
    rgba = point_cloud.copy_data("rgba")
    xyzrgba = point_cloud.copy_data("xyzrgba")
    rgb = rgba[..., :3]

print("xyz.shape:", xyz.shape)
print("rgb.shape:", rgb.shape)
print("xyzrgba.shape:", xyzrgba.shape)

xyz.shape: (1200, 1920, 3)
rgb.shape: (1200, 1920, 3)
xyzrgba.shape: (1200, 1920)


In [4]:
img_f = rawpy.imread(os.path.join(path, cam2)).postprocess(use_camera_wb=True, no_auto_bright=True, output_bps=8)[:8733, :11644]

# rotate xyz 180 degrees
xyz = xyz[::-1, ::-1, :]
rgb = rgb[::-1, ::-1, :]
xyz = xyz[:, 160:-160, :]
rgb = rgb[:, 160:-160, :]

In [5]:
# project xyz to img_f with k1 and RT
xyz_ = xyz.reshape(-1, 3)
# xyz_ = xyz_[xyz_[:, 2] > 0]
xyz_ = np.hstack((xyz_, np.ones((xyz_.shape[0], 1))))
xyz_f = RT @ xyz_.T

uv = k1 @ xyz_f
xyz_f = xyz_f.T.reshape(1200, 1600, 3)

uv = uv.T
uv = uv[:, :2] / uv[:, 2:]
uv = uv.reshape(1200,1600,2)

print(uv)

[[[          nan           nan]
  [          nan           nan]
  [          nan           nan]
  ...
  [          nan           nan]
  [          nan           nan]
  [          nan           nan]]

 [[          nan           nan]
  [          nan           nan]
  [          nan           nan]
  ...
  [3471.01958373 6984.45144948]
  [3467.12523327 6984.3409704 ]
  [3464.17381962 6983.77253182]]

 [[          nan           nan]
  [          nan           nan]
  [          nan           nan]
  ...
  [3471.06110365 6981.75109444]
  [3467.66160248 6981.46394578]
  [3463.94174505 6981.47099509]]

 ...

 [[          nan           nan]
  [8724.91004473 3036.44365578]
  [8721.84255509 3039.32937339]
  ...
  [3880.33072791 2957.21190894]
  [3877.15372145 2956.7589376 ]
  [3874.25242966 2956.77329587]]

 [[          nan           nan]
  [          nan           nan]
  [          nan           nan]
  ...
  [3880.44331653 2954.82347982]
  [3877.27474938 2954.69326925]
  [3874.21786574 2954.383891